In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from joblib import Parallel, delayed
import xarray as xr
import time
import os
import pygmt

import AbFab as af

%load_ext autoreload
%autoreload 2


In [ ]:
import pygmt
#age_da = xr.open_dataarray('/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.2m.nc')
#age_da = xr.open_dataarray('/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc')
age_da = pygmt.grdsample('/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc',
                         region='g', spacing='5m')

sed_da = pygmt.grdsample('/Users/simon/GIT/pyBacktrack/pybacktrack/bundle_data/sediment_thickness/GlobSed.nc',
                         region='g', spacing='5m')
                         #region='/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc')

age_da = af.extend_longitude_range(age_da).sel(lon=slice(-190,190))
sed_da = af.extend_longitude_range(sed_da).sel(lon=slice(-190,190))

sed_da = sed_da.where(np.isfinite(sed_da), 1.)
sed_da = sed_da.where(sed_da<1000., 1000.)


rand_da = age_da.copy()
rand_da.data = af.generate_random_field(rand_da.data.shape)



age_da = age_da.sel(lon=slice(-50,20), lat=slice(-30,0))
sed_da = sed_da.sel(lon=slice(-50,20), lat=slice(-30,0))
rand_da = rand_da.sel(lon=slice(-50,20), lat=slice(-30,0))


sed_da.plot(vmin=0,vmax=1000)
plt.show()
#rand_da.plot()

print(sed_da.shape)
print(age_da.shape)
print(rand_da.shape)


In [ ]:
full_ny,full_nx = age_da.shape

#nx = 360
#ny = 200
#x0 = 0
#y0 = 0
#seafloor_age = age_da[:200,:200]
#ny,nx = age_da.shape
#sediment_thickness = np.random.rand(ny, nx) * 5  # Example sediment thickness in meters

params = {
    'H': 200,       # Base rms height in meters
    'kn': 0.05,   # Characteristic width in km^-1 (normal to ridge)
    'ks': 0.25,   # Characteristic length in km^-1 (parallel to ridge)
    'D': 2.2       # Fractal dimension
}


start = time.time()



def process_bathymetry_chunk(coord, age_dataarray, sed_dataarray, rand_dataarray, chunksize, chunkpad):
    
    #print(coord, chunksize)
    
    chunk_age = age_dataarray[coord[0]:coord[0]+chunksize+chunkpad, 
                   coord[1]:coord[1]+chunksize+chunkpad]
    chunk_sed = sed_dataarray[coord[0]:coord[0]+chunksize+chunkpad, 
                       coord[1]:coord[1]+chunksize+chunkpad]
    chunk_random = rand_dataarray[coord[0]:coord[0]+chunksize+chunkpad, 
                           coord[1]:coord[1]+chunksize+chunkpad]
    
    #print(np.median(chunk_sed.data))
    #chunk_sed.plot()
    #plt.show()
    #plt.hist(chunk_sed.data.flatten())
    #plt.show()

    if np.all(np.isnan(chunk_age.data)):
        print('Empty Chunk')
        return chunk_age
    
    #ny,nx = chunk_age.shape
    
    # Generate the synthetic bathymetry
    synthetic_bathymetry = af.generate_bathymetry_spatial_filter(chunk_age.data, 
                                                                 chunk_sed.data, 
                                                                 params,
                                                                 chunk_random)

    return xr.DataArray(synthetic_bathymetry, 
                        coords=chunk_age.coords, name='z')[int(chunkpad/2):int(-chunkpad/2),int(chunkpad/2):int(-chunkpad/2)]
    

chunksize = 50
chunkpad = 20

chunkpad = int(2 * np.round(chunkpad / 2))

coords = np.meshgrid(np.arange(0,full_ny-1,chunksize), np.arange(0,full_nx-1,chunksize))
coords = list(zip(coords[0].flatten(), coords[1].flatten()))

num_cpus = 4
results = Parallel(n_jobs=num_cpus)(delayed(process_bathymetry_chunk)(coord, 
                                                                      age_da, 
                                                                      sed_da, 
                                                                      rand_da,
                                                                      chunksize, 
                                                                      chunkpad) for coord in coords)

#res = process_bathymetry_chunk(coords[22], age_da, sed_da, rand_da, chunksize, chunkpad)

print(time.time() - start)


#xr.combine_by_coords(results)

resss = [result for result in results if 0 not in result.shape]


fig,ax = plt.subplots(figsize=(30,16))
for res in resss:
    plt.pcolormesh(res.lon, res.lat, res.data, vmin=-0.5, vmax=0.5, cmap='magma')
#for res in resss:
#    plt.pcolormesh(res.lon[int(chunkpad/2):int(-chunkpad/2)], 
#                   res.lat[int(chunkpad/2):int(-chunkpad/2)], 
#                   res.data[int(chunkpad/2):int(-chunkpad/2),int(chunkpad/2):int(-chunkpad/2)], 
#                   vmin=-0.5, vmax=0.5)
#plt.pcolormesh(res.lon, res.lat, res.data, cmap='magma')
#plt.pcolormesh(resss[0].lon[:chunksize], resss[0].lat[:chunksize], resss[0].data[:chunksize,:chunksize], vmin=-0.5, vmax=0.5)
#plt.axis([-50,0,-30,0])
plt.colorbar()
plt.show()

In [ ]:
sed_da

In [ ]:
age_da